<a href="https://colab.research.google.com/github/appersaravanan/final_project/blob/master/God_grace1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from keras.applications import VGG16
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import  MaxPooling2D,UpSampling2D,Input,BatchNormalization,Conv2D,Activation,MaxPool2D,Flatten,Dense
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import VGG16
import cv2,os,tempfile
import requests
from keras.preprocessing import image
from scipy.io import loadmat
import tensorflow as tf
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [5]:
img_width, img_height = 224, 224
nb_train_samples = 8144
nb_validation_samples = 4899
epochs = 10
batch_size = 64
n_classes = 196

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content
!mkdir final_project
%cd ./final_project
!mkdir data
%cd data

Mounted at /content/drive
/content
/content/final_project
/content/final_project/data


In [4]:

!rm -f car_devkit.tgz
!wget https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
!tar xvzf car_devkit.tgz

--2020-12-06 16:36:07--  https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330960 (323K) [application/x-gzip]
Saving to: ‘car_devkit.tgz’

car_devkit.tgz      100%[===================>] 323.20K   258KB/s    in 1.3s    

2020-12-06 16:36:09 (258 KB/s) - ‘car_devkit.tgz’ saved [330960/330960]

devkit/
devkit/cars_meta.mat
devkit/cars_train_annos.mat
devkit/cars_test_annos.mat
devkit/README.txt
devkit/train_perfect_preds.txt
devkit/eval_train.m


In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_train.tgz

!tar xvzf cars_train.tgz
!rm -f cars_train.tgz


In [6]:
%cd /content/final_project/data

/content/final_project/data


In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_test.tgz
!tar xvzf cars_test.tgz
!rm -f cars_test.tgz

In [ ]:
%cd /content/final_project/data/cars_train/
annotsa = loadmat('/content/final_project/data/devkit/cars_meta.mat')
i=1
classes_name=[]
for name in annotsa['class_names'][0]:
  dir_name=str(i)+'_'+str(name[0]).replace(' ','_')
  os.makedirs(dir_name)
  classes_name.append(dir_name)
  i=i+1
classes_name


In [ ]:
%cd /content/final_project/data/cars_test/
annotsa = loadmat('/content/final_project/data/devkit/cars_meta.mat')
i=1
classes_name=[]
for name in annotsa['class_names'][0]:
  dir_name=str(i)+'_'+str(name[0]).replace(' ','_')
  os.makedirs(dir_name)
  if dir_name == '174_Ram_C/V_Cargo_Van_Minivan_2012':
    classes_name.append('174_Ram_C')
  else :
    classes_name.append(dir_name)
  i=i+1
classes_name

In [ ]:
%cd /content/final_project/data/cars_test/
annotsa = loadmat('/content/final_project/data/devkit/cars_meta.mat')
i=1
classes_name=[]
for name in annotsa['class_names'][0]:
  dir_name=str(i)+'_'+str(name[0]).replace(' ','_')
  os.makedirs(dir_name)
  if dir_name == '174_Ram_C/V_Cargo_Van_Minivan_2012':
    classes_name.append('174_Ram_C')
  else :
    classes_name.append(dir_name)
  i=i+1
classes_name

In [12]:
import shutil
from shutil import copyfile
path_base='/content/final_project/data'
train_data_dir = path_base + '/cars_train'
validation_data_dir = path_base + '/cars_test'


annos = loadmat('/content/final_project/data/devkit/cars_train_annos.mat')
data = [[row.flat[0] for row in line] for line in annos['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'fname']
df_train = pd.DataFrame(data, columns=columns)
gp=df_train.groupby('class')
files=""
for index,classes in gp:
    # print (classes_name[index-1])
    files=classes['fname']
    i=0
    for name,file in files.items():
        # print(file)
        copyfile('{}/{}'.format(train_data_dir,file),'{}/{}/{}'.format(train_data_dir,classes_name[index-1],file))
        if i<35:
          copyfile('{}/{}'.format(train_data_dir,file),'{}/{}/{}'.format(validation_data_dir,classes_name[index-1],file))
        i=i+1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
annos = loadmat('/content/final_project/data/devkit/cars_test_annos.mat')
data = [[row.flat[0] for row in line] for line in annos['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class']
df_train = pd.DataFrame(data)
df_train
# gp=df_train.groupby('class')
# files=""
# for index,classes in gp:
#     # print (classes_name[index-1])
#     files=classes['fname']
#     i=0
#     for name,file in files.items():
#         # print(file)
#         # copyfile('{}/{}'.format(train_data_dir,file),'{}/{}/{}'.format(train_data_dir,classes_name[index-1],file))
#         if i<25:
#           copyfile('{}/{}'.format(validation_data_dir,file),'{}/{}/{}'.format(validation_data_dir,classes_name[index-1],file))
#         i=i+1

In [13]:

# train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
#     .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(
    directory=train_data_dir,
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
    )

test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)



validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=10,
    class_mode='categorical',
    shuffle=True
    )

Found 8144 images belonging to 196 classes.
Found 6814 images belonging to 196 classes.


In [14]:

vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in enumerate(vgg16_model.layers):
        layer[1].trainable = False
for layer in vgg16_model.layers[15:]:
            layer.trainable = True
#flatten the results from conv block
x = Flatten()(vgg16_model.output)

#add another fully connected layers with batch norm and dropout
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.7)(x)

#add another fully connected layers with batch norm and dropout
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.7)(x)

#add logistic layer with all car classes
predictions = Dense(n_classes, activation='softmax', name='predictions')(x)

# this is the model we will train
model = Model(inputs=vgg16_model.input, outputs=predictions)




In [15]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# sgd = SGD(lr=0.0001, decay=lr_decay, momentum=0.9, nesterov=True)

In [8]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
callbacks_list = [early_stop, reduce_lr]

In [ ]:
model.summary()

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, mode='auto')
imagenette_builder = tfds.builder("cars196")
imagenette_info = imagenette_builder.info
imagenette_builder.download_and_prepare()
datasets = imagenette_builder.as_dataset(as_supervised=True)
train_data , test_data =datasets['train'],datasets['test']

In [ ]:
imagenette_info

In [12]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
      
  return tf.cast(tf.image.resize(image, (224, 224)), tf.float32) / 255.,  tf.one_hot(label,196)

ds_train = train_data.map(
   normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(imagenette_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test= test_data.map(
   normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# %cd /content/drive/'My Drive'/
# !mkdir checkpoints
# %cd checkpoints
# model.save_weights('./before_checkpoint1')
model.compile(
    optimizer='adam',    
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
# model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model_history = model.fit(
    ds_train, 
    validation_data=ds_test, 
    shuffle=True,
    epochs=100,
    batch_size=128
)

model.save_weights('./after_checkpoint')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/100
64/64 [==============================] - 40s 632ms/step - loss: 7.8199 - accuracy: 0.0082 - val_loss: 55.7784 - val_accuracy: 0.0063
Epoch 2/100
64/64 [==============================] - 40s 623ms/step - loss: 7.2746 - accuracy: 0.0147 - val_loss: 8.6580 - val_accuracy: 0.0177
Epoch 3/100
64/64 [==============================] - 42s 651ms/step - loss: 6.8385 - accuracy: 0.0182 - val_loss: 17.2088 - val_accuracy: 0.0093
Epoch 4/100
64/64 [==============================] - 41s 639ms/step - loss: 6.7566 - accuracy: 0.0176 - val_loss: 14.4547 - val_accuracy: 0.0088
Epoch 5/100
64/64 [==============================] - 40s 628ms/step - loss: 7.1301 - accuracy: 0.0163 - val_loss: 42.6391 - val_accuracy: 0.0095
Epoch 6/100
64/64 [==============================] - 40s 626ms/step - loss: 6.6790 - accuracy: 0.0238 - val_loss: 8.6294 - val_accuracy: 0.0189
Epo

In [ ]:

CLASS_INDEX={}
for i in range(len(classes_name)):
  CLASS_INDEX[str(i)]=[str(i),classes_name[i]]
print(CLASS_INDEX)
# from tensorflow.python.keras.utils import data_utils
# import json
# CLASS_INDEX= {
#             '0':  ['n01440764', 'tench'],
#             '1':  ['n02102040', 'English_springer'],
#             '2':  ['n02979186',  'casette_player'],
#             '3':  ['n03000684', 'chain_saw'],
#             '4':  ['n03028079',  'church'],
#             '5':  ['n03394916',  'French_horn'],
#             '6':  ['n03417042',  'garbage_truck'],
#             '7':  ['n03425413',  'gas_pump'],
#             '8':  ['n03445777',  'golf_ball'],
#             '9':  ['n03888257', 'parachute']
#         }
# # CLASS_INDEX_PATH = ('https://storage.googleapis.com/download.tensorflow.org/'
# #                     'data/imagenet_class_index.json')
# # def test_my_decode_predictions(*args, **kwargs):
# #     return my_decode_predictions(*args, **kwargs)


def my_decode_predictions(preds, top=5, **kwargs): 
  
    

    # print(CLASS_INDEX)
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [ ]:
test_images_array=[]
input_images_array=[]


img = image.load_img(train_data_dir+'/00073.jpg', target_size=(224, 224))
input_images_array.append(img)
img2 = image_utils.img_to_array(img)  
pre_processed_image = preprocess_input(img2);
test_images_array.append(pre_processed_image)

prediction_output_arr=[]
prediction_output_decoded_arr=[]
for i in range(len(test_images_array)): 
  prediction_result= model.predict(test_images_array[i][np.newaxis,...])  
  prediction_output=my_decode_predictions(prediction_result)
  # prediction_output_arr.append(prediction_result)
  # prediction_output_decoded_arr.append(prediction_output)
  print(prediction_output)

In [ ]:

from shutil import copyfile
path_base='/content/final_project/data'
train_data_dir = path_base + '/cars_train'
validation_data_dir = path_base + '/cars_test'


annos = loadmat('/content/final_project/data/devkit/cars_train_annos.mat')
data = [[row.flat[0] for row in line] for line in annos['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'fname']
df_train = pd.DataFrame(data, columns=columns)
df_train

In [ ]:
x=z

In [ ]:
predictions = model.predict(x=validation_generator, verbose=0)
np.round(predictions)

In [ ]:
def build_model():
    base_model = densenet.DenseNet121(input_shape=(img_width, img_height, 3),
                                     
                                     include_top=False,
                                     pooling='avg')
    for layer in base_model.layers:
      layer.trainable = True

    x = base_model.output
    x = Dense(1000, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = Activation('relu')(x)
    x = Dense(500, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = Activation('relu')(x)
    predictions = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

In [ ]:
model = build_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mse'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
callbacks_list = [early_stop, reduce_lr]

In [ ]:
model_history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
    # , callbacks=callbacks_list
    )